In [35]:
from pyscopus import Scopus
import pandas as pd
import requests
import xmltodict, json


from lxml import etree
import requests
import xmltodict, json

from bs4 import BeautifulSoup
import numpy as np



In [36]:
#print("Ingrese el dato a buscar:")
#busqueda = input()

Prueba Masiva 

In [37]:
 #Declaracion 
listalinks = list()
listatitulos = list()
listabstract = list()
listautor= list()
listakeywords = list()
listafuente = list()
listkey=list()

In [38]:
def consumo_scopus(busqueda):
    
    
    key = '0387c7142db3254ed53c09c131ecf8c0'
    scopus = Scopus(key)
    search_df = scopus.search("ALL("+busqueda+")", count=100)
    for i in range(len(search_df['scopus_id'])):
        try:
            pub_abs = scopus.retrieve_abstract(search_df['scopus_id'][i])
            listabstract.append(pub_abs['abstract'])
        except:
            listabstract.append("0")
            
        try:
            listalinks.append("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp="+search_df['scopus_id'][i]+"&origin=inward")
            
        except:
            listalinks.append("0")
            
        try:
            listatitulos.append(search_df['title'][i])
        except:
            listatitulos.append("0")
            
        try:
            list_aux=[]
            for j in range(len(search_df['authors'][i])):
                aut=scopus.retrieve_author(search_df['authors'][i][j])
                list_aux.append(aut['name'])
                
            listautor.append(list_aux)
        except:
            listautor.append("0")
            
        listakeywords.append("0")
        listafuente.append("scopus")
        listkey.append(busqueda)



In [39]:
def consumo_doaj(busqueda):
    #pasar a ingles 
    #funciona con el spacio 
    url = 'https://www.doaj.org/api/v2/search/articles/all('+busqueda+')?pageSize=100'
    headers = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r = requests.get(url, headers=headers)

    #Consumo api  de doaj sobre titulo,url,abstract,author(uspto),keywords
    content = r.json()
    #Ingreso al contenido
    json = content['results']

    for i in range(len(json)):
        try:
            listakeywords.append(json[i]['bibjson']['keywords'])
        except:
            listakeywords.append('0')
        try:
            listalinks.append(json[i]['bibjson']['link'][0]['url'])
        except:
            listalinks.append('0')
        try:
            listatitulos.append(json[i]['bibjson']['title'])
        except:
            listatitulos.append('0')
        try:
            listabstract.append(json[i]['bibjson']['abstract'])
        except:
            listabstract.append('0')
        
        try:
            aux_aut=[]
            for j in range(len(json[i]['bibjson']['author'])):
                aux_aut.append(json[i]['bibjson']['author'][0]['name'])
            listautor.append(list(set(aux_aut)))
        except:
            listautor.append('0')
            
        listafuente.append("doaj")
        listkey.append(busqueda)


In [40]:
def consumo_proquest(busqueda):

    import xmltodict, json
    access_token = '8461684a-5a79-407c-9321-93d8d719ddeb'
    headers = {
        'Content-type': 'text/xml',
        'Authorization': 'Bearer {}'.format(access_token)
    }
    data = '<searchRequest><search><query>'+busqueda+'</query><databases><database>medlineprof</database></databases></search><count>100</count></searchRequest>'
    resp = requests.post('https://api-dialog.proquest.com/v1/search', headers=headers, data=data)
    o = xmltodict.parse(resp.text)
    conta = json.dumps(o)
    d = json.loads(conta)
    

    json = d['searchResponse']['result']['doc']
    for i in range(len(json)):

            import re
            try:
                listatitulos.append(json[i]['field'][1]['value'])
                response2 = requests.get(json[i]['field'][4]['value'], headers=headers)
                o1 = xmltodict.parse(response2.text)
            except:
                listatitulos.append("0")
            
            try:
                listalinks.append(o1['Documents']['Literature']['DocInfo']['URL'])
            except:
                listalinks.append("0")
            try:
                nivel5= o1['Documents']['Literature']['Abstract']
                listabstract.append(nivel5.get('#text'))
            except:
                listabstract.append("0")
            try:
                contributor=o1['Documents']['Literature']['Contributors']['Contributor']
                aux=list()
                for i in range(len(contributor)):
                    if(contributor[i]['@ContribRole']=='Author'):
                        aux.append(contributor[i]['NormalizedName'])
                listautor.append(aux)
            except:
                listautor.append("0")
            try:
                keywords=o1['Documents']['Literature']['Subjects']['HeadingTerms']['HeadingTerm']
                aux=list()
                for i in range(len(keywords)):
                    nivel9=keywords[i]['Heading']
                    aux.append(nivel9['#text'])
                listakeywords.append(aux)
            except:
                listakeywords.append('0')    
            
            listafuente.append('ProQuest')
            listkey.append(busqueda)

In [41]:
def consumo_uspto(busqueda):
    #api lenta 
    #funciona con spacio 
    #ingle
    url_uspto = 'https://developer.uspto.gov/ibd-api/v1/application/publications?searchText='+busqueda+'&start=0&rows=100'
    headers_uspto = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r2_uspto = requests.get(url_uspto, headers=headers_uspto) 
    #Consumo api  de propiedad intelectual sobre titulo,url,abstract,nombre(uspto)
    content_uspto = r2_uspto.json()
    json_uspto = content_uspto['results']

    for i in range(len(json_uspto)):
        
        try:
            listatitulos.append(json_uspto[i]['inventionTitle'])
        except:
            listatitulos.append('0')
        try:
            listalinks.append(json_uspto[i]['filelocationURI'])
        except:
            listalinks.append('0')
        try:
            listabstract.append(json_uspto[i]['abstractText'][0])
        except:
            listabstract.append("0")
        try:
            listautor.append(json_uspto[i]['inventorNameArrayText'])
        except:
            listautor.append('0')
           
        listafuente.append("uspto")
        listakeywords.append(0) 
        listkey.append(busqueda)
            

In [42]:
def consumo_ieeexplore(busqueda):
    #https://www.google.co.uk/search?q=%22medicine%22&tbm=nws&gws_rd=ssl
    url_ieeexplore = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3dyyvg36jv78dbuu83q3fzzt&format=json&max_records=100&start_record=1&sort_order=asc&sort_field=article_number&querytext='+busqueda
    headers_ieeexplore = {'Content-Type': 'application/json;charset=UTF-8', 'Access-Control-Allow-Origin': '*'}
    r_ieeexplore = requests.get(url_ieeexplore, headers=headers_ieeexplore)

    content_ieeexplore = r_ieeexplore.json()
    json_ieeexplore = content_ieeexplore['articles']
    for i in range(len(json_ieeexplore)):
        try:
            listabstract.append(json_ieeexplore[i]['abstract'])
        except:
            listabstract.append('0')
        try:
            listatitulos.append(json_ieeexplore[i]['title'])
        except:
            listatitulos.append('0')
            
        try:
            listalinks.append(json_ieeexplore[i]['pdf_url'])
        except:
            listalinks.append("0")
        try:
            aux=list()
            for j in range(len(json_ieeexplore[i]['authors']['authors'])):
                aux.append(json_ieeexplore[i]['authors']['authors'][j]['full_name'])
            listautor.append(aux)
        except:
            listautor.append("0")
        try:
            listakeywords.append(json_ieeexplore[i]['index_terms']['ieee_terms']['terms'])
        except:
            listakeywords.append('0')  
        listafuente.append('ieeexplore')
        listkey.append(busqueda)
    

In [43]:
def consumo_google_noticas(busqueda):
    busqueda =busqueda.replace(" ", "+")
    res = requests.get('https://www.google.com/search?q='+busqueda+'+&biw=1064&bih=1034&tbm=nws&sxsrf=ALeKk012okzl46BThz3OHzeulw_CfNeALg%3A1620065628921&ei=XD2QYN2hN52y5NoP88anwAw&oq='+busqueda+'+&gs_l=psy-ab.3...1712.7789.0.7910.36.29.2.0.0.0.336.3435.0j13j5j1.20.0....0...1c.1.64.psy-ab..15.11.1630.0..0.124.YgObgGkRCac')
    soup = BeautifulSoup(res.text, 'lxml')
    
    title_noticias = soup.find_all('div', class_="BNeawe vvjwJb AP7Wnd")
    for i in range(len(title_noticias)):
        print('titulo')
        print(title_noticias[i].text)
        listatitulos.append(title_noticias[i].text)
    
    print(soup.find_all('div', class_="BNeawe s3v9rd AP7Wnd"))
    primer=soup.find_all('div', class_="BNeawe s3v9rd AP7Wnd")
    print(primer)



   

In [44]:


def categorizador(listatitulos,listabstract,listakeywords):
    
    import classifier.classifier as CSO

    list_clasificador=[]
    for i in range(len(listatitulos)):
        try:
            result = CSO.run_cso_classifier(listatitulos[i]+" "+listabstract[i]+" "+','.join(listakeywords[i]), modules="both", enhancement = "first")
            list_clasificador.append(list(set(result.get('syntactic')+result.get('semantic')+result.get('union')+result.get('enhanced'))))
        except:
            result = CSO.run_cso_classifier(listatitulos[i]+" "+listabstract[i], modules="both", enhancement = "first")
            list_clasificador.append(list(set(result.get('syntactic')+result.get('semantic')+result.get('union')+result.get('enhanced'))))
                  
    diccionario ={'link':listalinks ,'title':listatitulos,'abstract':listabstract,'authors':listautor,"keywords":listakeywords,'fuente':listafuente,'categoria':list_clasificador,'key':listkey}
    
    print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente),'cAtegoria',len(list_clasificador))

    
    return diccionario

In [45]:

dataset_palabras = pd.read_csv('palabras.csv')

#print(dataset_palabras['n.rdfs__label'])
list_palabras_error=[]
for i in range(5):
    print("Procesamiento:",i)
    print("Palabra:",dataset_palabras['n.rdfs__label'][i])
    busqueda = dataset_palabras['n.rdfs__label'][i]
    try:
        consumo_proquest(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])
    try:
        consumo_scopus(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])

    try:
        consumo_doaj(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])


    try:
        consumo_uspto(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])


    try:
        consumo_ieeexplore(busqueda)
        print('link',len(listalinks),' title ',len(listatitulos),'abstract ',len(listabstract),' authors ',len(listautor)," keywords ",len(listakeywords),'fuente', len(listafuente))
    except:
        list_palabras_error.append(dataset_palabras['n.rdfs__label'][i])
    
    

Procesamiento: 0
Palabra: Ullrich congenital muscular dystrophy
link 7  title  7 abstract  7  authors  7  keywords  7 fuente 7
link 107  title  107 abstract  107  authors  107  keywords  107 fuente 107
link 117  title  117 abstract  117  authors  117  keywords  117 fuente 117
link 217  title  217 abstract  217  authors  217  keywords  217 fuente 217
link 219  title  219 abstract  219  authors  219  keywords  219 fuente 219
Procesamiento: 1
Palabra: obsolete JMP syndrome
link 224  title  224 abstract  224  authors  224  keywords  224 fuente 224
link 224  title  224 abstract  224  authors  224  keywords  224 fuente 224
link 224  title  224 abstract  224  authors  224  keywords  224 fuente 224
Procesamiento: 2
Palabra: obsolete Saldino-Noonan syndrome
link 225  title  225 abstract  225  authors  225  keywords  225 fuente 225
link 225  title  225 abstract  225  authors  225  keywords  225 fuente 225
link 225  title  225 abstract  225  authors  225  keywords  225 fuente 225
Procesamiento: 3

In [46]:
dic_general=categorizador(listatitulos,listabstract,listakeywords)

Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and cached model loaded.
Computer Science Ontology and c

In [47]:

dataset=pd.DataFrame(dic_general)
print(dataset)
dataset.to_csv('ResultadoCategoria.csv')

                                                  link  \
0    http://dialog.proquest.com/professional/docvie...   
1    http://dialog.proquest.com/professional/docvie...   
2    http://dialog.proquest.com/professional/docvie...   
3    http://dialog.proquest.com/professional/docvie...   
4    http://dialog.proquest.com/professional/docvie...   
5    http://dialog.proquest.com/professional/docvie...   
6    http://dialog.proquest.com/professional/docvie...   
7    https://www.scopus.com/inward/record.uri?partn...   
8    https://www.scopus.com/inward/record.uri?partn...   
9    https://www.scopus.com/inward/record.uri?partn...   
10   https://www.scopus.com/inward/record.uri?partn...   
11   https://www.scopus.com/inward/record.uri?partn...   
12   https://www.scopus.com/inward/record.uri?partn...   
13   https://www.scopus.com/inward/record.uri?partn...   
14   https://www.scopus.com/inward/record.uri?partn...   
15   https://www.scopus.com/inward/record.uri?partn...   
16   https://w